reference 

- https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system#Collaborative-Filtering
- https://realpython.com/build-recommendation-engine-collaborative-filtering/#memory-based

In [4]:
import pandas as pd 
import content_based_user_rec

def get_similar_users(user_id):
    pha_user= pd.read_csv('pha_user.csv')
    pha_user = pha_user[pha_user['is_superuser'] != True]
    recommendation_dic = dict()
    n_user = len(list(pha_user['us_id']))
    for i in range(0, n_user-1):
        cur_user = pha_user['us_id'].iloc[i]
        result = content_based_user_rec.get_recommendations(cur_user)
        recommendation_dic[cur_user] = result

    
    result = recommendation_dic[user_id]
    return result

In [24]:
# 5 similar users 
similar_user_list = get_similar_users(2)[0:9]
similar_user_list.append(2)

import psycopg2

# rating table = food_id, user_id, rating, meal_time (latest update time)
# need to get the project_id and user_id 
def get_rating_table(similar_user_list, project_id, user_id):
    similar_user_tuple = tuple(similar_user_list)
    conn = psycopg2.connect(
        host = 'localhost', # find it from my_setting.spy in HealthGeinie directory
        database = 'postgres',
        user = 'postgres',
        password = '0847'
    )
    cur = conn.cursor()
    query = "select temp.food_id, temp.user_id, temp.rating, temp.meal_time from (select food_id, user_id, rating, meal_time, meal_id from new_pha_meal where user_id IN %s ORDER BY food_id, user_id, meal_time) as temp where temp.meal_time <= (select end_time from pha_project where project_id = %s and user_id = %s) and temp.meal_time >= (select start_time from pha_project where project_id = %s and user_id = %s) AND temp.meal_time = (select MAX(meal_time) FROM new_pha_meal WHERE food_id = temp.food_id AND user_id = temp.user_id);"
    cur.execute(query, (similar_user_tuple, project_id,user_id,project_id,user_id,))
    result = cur.fetchall()
    cur.close()
    return result

food_rating = get_rating_table(similar_user_list, 5, 2)
#print(food_rating[0])

In [25]:
similar_user_list

[4, 6, 11, 10, 5, 3, 12, 8, 2, 2]

In [ ]:
# if it cannot be built, you should check On Windows, you may need to install Microsoft Visual C++ Build Tools or Microsoft Visual Studio.
#pip install scikit-surprise

In [26]:
import pandas as pd
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import train_test_split

# Load the dataset
#data = Dataset.load_builtin('ml-100k')
reader = Reader(rating_scale=(1, 5))
rating = pd.DataFrame(food_rating)
rating.columns = ['food_id', 'user_id', 'rating', 'update_time']
data = Dataset.load_from_df(rating[['user_id', 'food_id', 'rating']], reader)


# Split the dataset into training and testing sets
trainset, testset = train_test_split(data, test_size=.25)

# Train the model using SVD algorithm
algo = SVD()
algo.fit(trainset)

# Get the list of all unique user ids
user_ids = data.df['user_id'].unique()

# Create a dictionary to store the top recommended items for each user
top_recs = {}

# For each user id, get the list of items that have not been rated by the user
for user_id in user_ids:
    # Get the list of all items
    items = data.df['food_id'].unique()
    # Get the list of items that have been rated by the user
    rated_items = data.df[data.df['user_id'] == user_id]['food_id'].unique()
    # Get the list of items that have not been rated by the user
    unrated_items = list(set(items) - set(rated_items))
    # Get the predicted ratings for the unrated items
    predicted_ratings = [algo.predict(user_id, item_id).est for item_id in unrated_items]
    # Combine the list of unrated items with their predicted ratings
    unrated_items_with_ratings = zip(unrated_items, predicted_ratings)
    # Sort the items by their predicted ratings
    sorted_items = sorted(unrated_items_with_ratings, key=lambda x: x[1], reverse=True)
    # Get the top recommended items for the user
    top_items = [item_id for item_id, rating in sorted_items[:10]]
    # Add the top recommended items to the dictionary
    top_recs[user_id] = top_items
    print(top_recs[user_id])


[]
[]
[34, 20]
[]
[]
[43]
[]
[]
[]


In [27]:
top_recs

{2: [], 3: [], 4: [34, 20], 5: [], 6: [], 8: [43], 10: [], 11: [], 12: []}